# CIFAR-10


In [1]:
import torch
from torchvision import datasets


# 데이터 불러오기: torchvision 패키지에서 cifar10의 train data 와 test data를 불러옴
def load_data(dataset, dataset_path):
    train_dataset = None
    test_dataset = None

    if dataset == "CIFAR10":
        train_dataset = datasets.CIFAR10(
            dataset_path,  # 해당 데이터 셋의 path
            download=True,  # 해당 데이터셋을 다운 받을 것인지
            train=True,
        )  # 학습 용도로 사용될 것인지
        train_dataset.data = torch.tensor(
            train_dataset.data
        )  # numpy array 형태를 torch tensor로 변경
        train_dataset.data = torch.permute(
            train_dataset.data, dims=(0, 3, 1, 2)
        )  # (B,H,W,C) -> (B,C,H,W) 형태로 변경
        train_dataset.targets = torch.tensor(train_dataset.targets)

        test_dataset = datasets.CIFAR10(dataset_path, download=False, train=False)
        test_dataset.data = torch.tensor(test_dataset.data)
        test_dataset.data = torch.permute(test_dataset.data, dims=(0, 3, 1, 2))
        test_dataset.targets = torch.tensor(test_dataset.targets)

    else:
        print("Incorrect dataset!")
    return train_dataset, test_dataset

In [ ]:
# Input Preprocess: Model이 학습할 수 있도록 input으로 들어가는 train data를 수정함
def input_preprocess(x, train_mode=True):
    x = x / 255.0  # MinmaxScaler : input data 를 0~1사이의 float로 바꿔주기
    if train_mode:
        half = int(x.shape[0] / 2)  # 미니배치 개수의 반
        x[0:half, :] = torch.flip(x[0:half, :], dims=[3])  # (B,C,H,W)의 W를 반전

    return x

In [ ]:
def onehot_encoding(y, n_class):
    out = torch.zeros(
        len(y), n_class
    )  # 전체 클래스 수 만큼 원소를 갖는 zero vector 들을 생성
    for i in range(len(y)):  # 미니배치 수(y의 길이) 만큼 반복
        out[i, y[i]] = 1  # target index를 1로 채워두기
    y = out.float()  # 추후 연산을 위해서 dtype을 float으로 변환
    return y

# OpenCV에서 이미지 불러오기


In [ ]:
import cv2

input_image = cv2.imread("image.png")
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)  # BGR -> RGB
input_image = torch.Tensor(input_image)  # Numpy array를 torch Tensor로 변환
input_image = torch.permute(input_image, (2, 0, 1))  # (H,W,C) -> (C,H,W)
input_image = torch.unsqueeze(input_image, 0)  # (C, H, W) -> (1, C, H, W)